In [ ]:
import requests
import time
from fair_research_login import NativeClient

client = NativeClient(client_id='7414f0b4-7d05-4bb6-bb00-076fa3f17cf5')
tokens = client.login(
    requested_scopes=['urn:globus:auth:scope:transfer.api.globus.org:all',
                      "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all", 
                      'email', 'openid'],
    no_local_server=True,
    no_browser=True)

transfer_token = tokens['transfer.api.globus.org']['access_token']
funcx_token = tokens['funcx_service']['access_token']
headers = {'Authorization': f"Bearer {funcx_token}",'Transfer': transfer_token, 'FuncX': f"{funcx_token}"}
print(f"Headers: {headers}")

In [ ]:
t0 = time.time()
# Example for posting a file to the application
url = "http://127.0.0.1:5000/upload_def_file"
file_name = "Dockerfile"
file_path = "Dockerfile"
my_files = {"file": (file_name, open(file_path,"rb"))}
response = requests.post(url, files=my_files, headers=headers)
definition_id = response.text
print(definition_id)
print(time.time() - t0)

In [ ]:
t0 = time.time()
# Example for building a container
url = "http://127.0.0.1:5000/build"
my_data = {"definition_id": definition_id, "to_format": "docker", 
           "container_name": "my_container"}
response = requests.post(url, json=my_data, headers=headers)
build_id = response.text
print(build_id)
print(time.time() - t0)

In [ ]:
t0 = time.time()
# Example for pulling a container
url = "http://127.0.0.1:5000/pull"
my_data = {"build_id": build_id}
response = requests.get(url, json=my_data, headers=headers)
print(response)
with open("my_container.tar", 'wb') as f:
    f.write(response.content)
print(response.content)
print(time.time() - t0)